In [1]:
# yelp scraping project
import requests
import json
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import re

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 2500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [3]:
# hard code this section to scrape multiple pages for reviews
main = 'https://www.yelp.com/biz/made-man-salon-tustin'
url = [main,
       main + '?start=20',]
      #main + '?start=40',
      #main + '?start=60',
      #main + '?start=80',
      #main + '?start=100',
      #main + '?start=120',
      #main + '?start=140',
      #main + '?start=160',]
      #main + '?start=180',
      #main + '?start=200',
      #main + '?start=220',
      #main + '?start=240',
      #main + '?start=260',
      #main + '?start=280',
      #main + '?start=300',]

In [4]:
for i in url:
    print (i)

https://www.yelp.com/biz/made-man-salon-tustin
https://www.yelp.com/biz/made-man-salon-tustin?start=20


# Start Review Section

In [5]:
reviews = []

In [6]:
def get_review_content(url):
    for i in url:
        response = requests.get(i)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        rc_unstripped = soup.find_all(attrs={'class': 'review-content'})
        rc_length = (len(rc_unstripped))
        
        for i in range (0, rc_length):
            reviews.append(rc_unstripped[i].text.strip())       

In [7]:
# Run the function
get_review_content(url)

In [8]:
print(len(reviews))

23


In [9]:
print(reviews[2])

9/9/2017
    

A-M-A-Z-I-N-GThis place is simply amazing, Yelena is incredible talented, passionate about her business and an incredible human being. I discovered Yelena last year and since then I don't let anybody to cut my hair. She gives an ultimate upscale 5 stars attention to her clients from the moment you walk in to the moment you leave. She's simple a ROCK STAR!!!!!!!!!!! she's smart and her skills are out of this world. I always say: Once Yelena, you can't leave her. If I could give her more than 5 stars I would totally do it. She deserves more than that.


In [10]:
# convert reviews to pd series
reviews_s = pd.Series(reviews)
reviews_s.head()

0    1/24/2018\n    \n\nI think I've finally found ...
1    10/14/2017\n    \n\nI've been a client with Ye...
2    9/9/2017\n    \n\nA-M-A-Z-I-N-GThis place is s...
3    2/2/2017\n    \n\n\n\n\n\n\n\n\n7 check-ins\n ...
4    3/3/2017\n    \n\nI've been in Orange County f...
dtype: object

In [11]:
# create a pandas df from reviews
data = {'Reviews' : reviews_s}

df = pd.DataFrame.from_dict(data)
df = df[['Reviews']]    

In [12]:
df.head()

,Reviews
0,1/24/2018\n \n\nI think I've finally found ...
1,10/14/2017\n \n\nI've been a client with Ye...
2,9/9/2017\n \n\nA-M-A-Z-I-N-GThis place is s...
3,2/2/2017\n \n\n\n\n\n\n\n\n\n7 check-ins\n ...
4,3/3/2017\n \n\nI've been in Orange County f...


In [13]:
# split date and reviews
df['Date'] = df.Reviews.str.split('\n').str.get(0)
df['Review'] = df.Reviews.str.split('\n\n').str.get(1)

In [14]:
# drop Reviews column
df.drop('Reviews', axis=1, inplace=True)

In [15]:
df.head()

,Date,Review
0,1/24/2018,I think I've finally found THE best place in O...
1,10/14/2017,I've been a client with Yelena for almost 12 y...
2,9/9/2017,"A-M-A-Z-I-N-GThis place is simply amazing, Yel..."
3,2/2/2017,
4,3/3/2017,I've been in Orange County for 7 years now and...


In [16]:
# delete NaN rows
df2 = df.replace(r"", np.nan)
df2.tail()

,Date,Review
18,2/12/2017,Yelena has been my go to stylist for many year...
19,2/4/2017,"I have been going to Yelena for 2 years now, a..."
20,5/6/2017,First visitors. We made it a family event. My...
21,4/27/2017,Yelena has been my stylist for nearly a decade...
22,3/9/2017,Yelena is the best in the business. I've been ...


In [17]:
# drop NaN rows
df3 = df2.dropna()

# reset index
df3.reset_index(drop=True, inplace=True)

df3.head()
df3.shape

,Date,Review
0,1/24/2018,I think I've finally found THE best place in O...
1,10/14/2017,I've been a client with Yelena for almost 12 y...
2,9/9/2017,"A-M-A-Z-I-N-GThis place is simply amazing, Yel..."
3,3/3/2017,I've been in Orange County for 7 years now and...
4,3/3/2017,Yelena has my complete trust when styling my h...


(21, 2)

In [18]:
# checks to see if there are any null values
df3.isnull().sum()

Date      0
Review    0
dtype: int64

# Pandas DF and Sentiment Section

In [19]:
from textblob import TextBlob

In [20]:
# convert Review column in df3 to list
review_list = df3['Review'].tolist()
review_list[0]

"I think I've finally found THE best place in OC to get my hair cut! \xa0Finally! \xa0Awesome cut, awesome experience. Normally it takes going to a hair stylist for a while to get it right-- nailed it on my first visit."

In [21]:
len(review_list)

21

In [22]:
sentiment_score = []
# iterate through review list
for r in review_list:
    sentiment = TextBlob(r)
    sentiment_score.append(sentiment.sentiment.polarity)

In [23]:
# convert to series
sentiment_score_convert = pd.Series(sentiment_score)

In [24]:
# add sentiment score to sentiment_df
df3.insert(loc=1, column='SentimentScore', value=sentiment_score_convert)

In [25]:
df3.head()

,Date,SentimentScore,Review
0,1/24/2018,0.526531,I think I've finally found THE best place in O...
1,10/14/2017,0.324125,I've been a client with Yelena for almost 12 y...
2,9/9/2017,0.328022,"A-M-A-Z-I-N-GThis place is simply amazing, Yel..."
3,3/3/2017,0.450617,I've been in Orange County for 7 years now and...
4,3/3/2017,0.360000,Yelena has my complete trust when styling my h...


In [26]:
sentiment = []
# iterate through sentiment_score to assign a pos or neg val
for s in sentiment_score:
    if s > 0:
        sentiment.append('Positive')
    else:
        sentiment.append('Negative')

In [27]:
print(len(sentiment_score))
print(len(sentiment))

21
21


In [28]:
# check for null values
df3.isnull().sum()

Date              0
SentimentScore    0
Review            0
dtype: int64

In [29]:
# convert sentiment to a pandas series
sentiment_2 = pd.Series(sentiment)

In [30]:
# add sentiment score to sentiment_df
df3.insert(loc=2, column='Sentiment', value=sentiment_2)

In [31]:
df3.head()

,Date,SentimentScore,Sentiment,Review
0,1/24/2018,0.526531,Positive,I think I've finally found THE best place in O...
1,10/14/2017,0.324125,Positive,I've been a client with Yelena for almost 12 y...
2,9/9/2017,0.328022,Positive,"A-M-A-Z-I-N-GThis place is simply amazing, Yel..."
3,3/3/2017,0.450617,Positive,I've been in Orange County for 7 years now and...
4,3/3/2017,0.360000,Positive,Yelena has my complete trust when styling my h...


In [32]:
# check value counts for Positive and Negative
df3.Sentiment.value_counts()

Positive    21
Name: Sentiment, dtype: int64

In [33]:
# convert Sentiment label to a numerical value
df3['SentimentNum'] = df3.Sentiment.map({'Positive': 0,
                                                 'Negative': 1})

In [34]:
df3.head()
df3.tail()
df3.shape

,Date,SentimentScore,Sentiment,Review,SentimentNum
0,1/24/2018,0.526531,Positive,I think I've finally found THE best place in O...,0
1,10/14/2017,0.324125,Positive,I've been a client with Yelena for almost 12 y...,0
2,9/9/2017,0.328022,Positive,"A-M-A-Z-I-N-GThis place is simply amazing, Yel...",0
3,3/3/2017,0.450617,Positive,I've been in Orange County for 7 years now and...,0
4,3/3/2017,0.360000,Positive,Yelena has my complete trust when styling my h...,0


,Date,SentimentScore,Sentiment,Review,SentimentNum
16,2/12/2017,0.010417,Positive,Yelena has been my go to stylist for many year...,0
17,2/4/2017,0.306169,Positive,"I have been going to Yelena for 2 years now, a...",0
18,5/6/2017,0.325000,Positive,First visitors. We made it a family event. My...,0
19,4/27/2017,0.137500,Positive,Yelena has been my stylist for nearly a decade...,0
20,3/9/2017,0.314719,Positive,Yelena is the best in the business. I've been ...,0


(21, 5)

In [35]:
df3.SentimentNum.value_counts()

0    21
Name: SentimentNum, dtype: int64

In [36]:
# extract business name from "main" url
bizname_split = main.split('https://www.yelp.com/biz/')
bizname = bizname_split[1]
bizname

'made-man-salon-tustin'

In [37]:
# save csv to reviews folder to use for NLP
df3.to_csv(f'reviews/{bizname}.csv', index=False)

# Section for NLTK

### Import saved csvs from reviews directory

In [ ]:
# Define X and y for use with COUNTVECTORIZE
X = df3.Review
y = df3.SentimentNum
print(X.shape)
print(y.shape)